# Import Packages

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import datetime

# Produce List Of Dates For Webscrape

In [ ]:
DateList = []
temp = datetime.date(1958,8,4)
DateList.append(temp) 
while temp < datetime.date.today():
    temp = temp + datetime.timedelta(days=7)
    DateList.append(str(temp))
print(DateList[1])

# Define Function To Pull Data & Organise In To Data Frame

In [ ]:
def Scrape(Date,Website):
    r=requests.get(str(Website))
    Data = BeautifulSoup(r.text,"html.parser")
    ArtistList = []
    for i in Data.find_all(class_="chart-list-item__artist"):
        ArtistList.append(re.search(".*(?=\n\s*</a>)|.*(?=[^</a>]</div>)",str(i))[0])
    TrackList = []
    for i in Data.find_all(class_="chart-list-item__title-text"):
        TrackList.append(re.search(".*(?=\n*</span>)",str(i))[0])
    Output = pd.DataFrame({"Track": TrackList,
                           "Artist": ArtistList,
                           "Rank": list(range(1, len(TrackList)+1)),
                           "Date": str(Date)})
    return(Output)

# Run Function For All Dates & Store In List

In [ ]:
DataFrameList = []
for j, i in enumerate(DateList):
    try:
        DataFrameList.append(Scrape(str(i),"https://www.billboard.com/charts/hot-100/"+str(i)+"/"))
        print(str(i)+" - "+str(j)+" Out Of ",str(len(DateList)))
    except:
        print(str(i) + "- Error")
        pass

# Join All Data Frames Together

In [ ]:
Data = DataFrameList[0]
for j, i in enumerate(DataFrameList):
    Data = pd.concat([Data,i])
    print(j)

# Output Data Frame As CSV

In [ ]:
Data.shape
Data.to_csv("TopChartSongs.csv", index = False)